In [14]:
import torch
def t(value):
    return torch.Tensor([value]).requires_grad_(True)

a=t(343.212)
b=t(543)
c=t(5.3)
d=t(54.1)

erro = (5+a)*b+(c*d/a)

erro.backward()
c.grad

tensor([0.1576])

In [ ]:
import math
class Drv:
    def __init__(self,num):
        self.num=num
        self.grad=0
        self.ant = ()
        self._backward = lambda: None

    def __str__(self):
        return f"num: {self.num} grad: {self.grad} anteriores {[a.num for a in self.ant]}"
    
    def __add__(self,outro):
        novo = Drv(self.num+outro.num)
        novo.ant = (self,outro)
        def drv_adicao():
            self.grad+=1*novo.grad
            outro.grad+=1*novo.grad
        novo._backward = drv_adicao
        return novo
    
    def __mul__(self,outro):
        novo = Drv(self.num*outro.num)
        novo.ant = (self,outro)
        def drv_mult():
            self.grad+=outro.num*novo.grad
            outro.grad+=self.num*novo.grad
        novo._backward = drv_mult
        return novo
    
    def sig(self):
        x = self.num
        
        if(x>15):
            sigx=0.99999
        elif(x<-15):
            sigx=0.00001
        else:
            sigx = 1/1+math.exp(-x)

        novo = Drv(sigx)
        novo.ant=(self,)
        def drv_sig():
            self.grad+= sigx*(1-sigx)*novo.grad
        novo._backward=drv_sig
        return novo

    def setar_derivadas_em_todos_os_anteriores(self):
        ordem_topo=[]
        visitados=set()
        def construir_ordem_topo(no:Drv):
            if no not in visitados:
                visitados.add(no)
                for var in no.ant:
                    construir_ordem_topo(var)
                ordem_topo.append(no)
        construir_ordem_topo(self)

        self.grad=1
        for dr in reversed(ordem_topo):
            dr._backward()
    

num: 43.6 grad: 0 anteriores [42.4, 1.2]
2
